In [41]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
import tensorflow as tf
from datetime import datetime
import cv2
import glob
tf.__version__

'2.3.0'

In [35]:
from keras_retinanet import models

def create_model(backbone_name, num_classes=1):
    backbone_factory = models.backbone(backbone_name)
    model = backbone_factory.retinanet(num_classes)
    return models.convert_model(model)

backbone = 'resnet50'
model = create_model(backbone)

In [36]:
from keras.layers import Input
from keras.models import Model

fixed_input = Input((512,512,3))
fixed_model = Model(fixed_input,model(fixed_input))
converter = tf.lite.TFLiteConverter.from_keras_model(fixed_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite = converter.convert()

with open("model.tflite","wb") as h:
    h.write(tflite)

In [37]:
interpreter = tf.lite.Interpreter(model_path='model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'input_11', 'index': 0, 'shape': array([  1, 512, 512,   3], dtype=int32), 'shape_signature': array([ -1, 512, 512,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 566, 'shape': array([1, 1, 1], dtype=int32), 'shape_signature': array([-1, -1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'Identity_1', 'index': 568, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'qu

{'name': 'input_11',
 'index': 0,
 'shape': array([  1, 512, 512,   3], dtype=int32),
 'shape_signature': array([ -1, 512, 512,   3], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [38]:
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
original_image = cv2.imread('test_data/img/1/0___n04759.jpg')
image_data = cv2.resize(original_image, (512,512))
image_data = preprocess_image(image_data)
#image_data = image_data / 255.
image_data = np.expand_dims(image_data, axis=0)
interpreter.allocate_tensors()

In [40]:
start = datetime.now()
interpreter.set_tensor(input_details[0]['index'], image_data)
interpreter.invoke()
pred = [interpreter.get_tensor(output_details[i]['index']) for i in range(len(output_details))]
end = datetime.now()
print(end- start)

0:00:00.541902


In [44]:
def representative_data_gen():
    fimage = glob.glob('test_data/img/1/*.jpg')
    for input_value in range(10):
        if os.path.exists(fimage[input_value]):
            original_image=cv2.imread(fimage[input_value])
            image_data = cv2.resize(original_image, (512,512))
            image_data = preprocess_image(image_data)
            img_in = image_data[np.newaxis, ...].astype(np.float32)
            print("calibration image {}".format(fimage[input_value]))
            yield [img_in]
        else:
            continue

In [45]:
from keras_retinanet import models

def create_model(backbone_name, num_classes=1):
    backbone_factory = models.backbone(backbone_name)
    model = backbone_factory.retinanet(num_classes)
    return models.convert_model(model)

backbone = 'resnet50'
model = create_model(backbone)

fixed_input = Input((512,512,3))
fixed_model = Model(fixed_input,model(fixed_input))
converter = tf.lite.TFLiteConverter.from_keras_model(fixed_model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.allow_custom_ops = True
converter.representative_dataset = representative_data_gen

tflite = converter.convert()

with open("model_int8.tflite","wb") as h:
    h.write(tflite)

ValueError: Failed to parse the model: pybind11::init(): factory function returned nullptr.